### Examples:

1. Predict the parent process name from the child process name

### Methodology:

1. Create "unseen", and "seen" data sets using data you created earlier, or new captures
1. Learn the relationship between process names and parent process names
1. Explore the effectiveness of the trained model on the unseen data set

### Try for yourself:

1. Predict the working directory from the process name
1. Predict the time of day from the child process name

In [ ]:
import matplotlib
import pandas
import psutil
import time

from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
columns = ['pid', 'ppid', 'cwd', 'create_time', 'cmdline', 'name']

In [ ]:
i = 0
vvalues = []

# Every 5 seconds for half an hour
for i in range(360):
    for proc in psutil.process_iter(attrs=columns):
        values = list(proc.info.values())
        vvalues += [values]
    
    time.sleep(5)


In [ ]:
df = pandas.DataFrame(columns=columns, data=vvalues)

In [ ]:
df['create_time'] = pandas.to_datetime(df['create_time'], unit='s')
df

In [ ]:
df.to_pickle('my_processes.pickle')

In [ ]:
len(df)

In [ ]:
names = set(df['name'])
len(names)

In [ ]:
list(names)[:5]

In [ ]:
name_to_index = {name: index for index, name in enumerate(names)}
index_to_name = {index: name for index, name in enumerate(names)}

In [ ]:
list(name_to_index.items())[:5]

In [ ]:
index_to_name[4]

## Our Goal

Create a dictionary showing the child process name, the parent process name, and the number of times that parent was observed to launch the child.

```
{ 
   (name1, name2): 3,
   (name1, name4): 1,
   (name5, name2): 2
   ...
}
```

But we really need to train our model on:

```
{ 
   (name_id_1, name_id_2): 3,
   (name_id_1, name_id_4): 1,
   (name_id_5, name_id_2): 2
   ...
}
```

A deeper, more complex goal might be:

```
{ 
   (name1, name2, name3): 3,
   (name1, name4, name5): 1,
   (name5, name2, name5): 2
   ...
}
```

Each sequence is like a sentence. The length of the sequence used for learning is an "n-gram"

In this way, by sequencing the IDs of "anything of note" with a causaul relationship, we can learn the prevalance of a causal chain, and start to perform machine learning over these event sequences in the same way as for word sequences.


In [ ]:
df['name_id'] = df.apply(lambda x: name_to_index[x['name']], axis=1)

In [ ]:
len(set(df['name_id']))

In [ ]:
pid_name_pair = df.apply(lambda x: (x['pid'], x['name_id']), axis=1)
pid_to_name_id = {pid: name for (pid, name) in list(pid_name_pair)}
pid_to_name_id[998]  

In [ ]:
index_to_name[221]  # expect bash 

In [ ]:
train, test = train_test_split(df, test_size=0.2)
train.head()